In [1]:
%matplotlib agg

#
import matplotlib
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath('/home/cat/code/donatolab/')
sys.path.append(module_path)

#from utils.wheel import wheel
#from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-156d6bfde458>:14: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
import cv2
from tqdm import trange

# Path to the TIFF file
root_dir = "/media/cat/4TB/donato/insular_cortex/"
animal_id = 'DON-015508'

animal_ids = [
    
    #'DON-015506',
    #'DON-015507',
    #'DON-015508',
    'DON-015509',
    'DON-015510',
    'DON-015512',
    
]

for animal_id in animal_ids:
    tiff_file = os.path.join(root_dir,
                             animal_id,
                             'data',
                             'Image_001_001.tiff')

    #                         
    import matplotlib.pyplot as plt
    import imageio
    from tifffile import TiffFile
    from scipy.ndimage import gaussian_filter

    # Open the tiff file
    try:
        tiff = TiffFile(tiff_file)
    except:
        print ("Couldn't find tiff file")

    # Get the image stack
    stack = tiff.asarray()

    # Create an imageio writer
    writer = imageio.get_writer(tiff_file.replace('.tiff','.gif'), fps=5)

    #
    sigma = [1, 1, 0]

    # Apply the Gaussian filter

    # Loop through the slices
    for i in trange(0, stack.shape[0], 30):

        # Extract the slice
        temp = stack[i:i+30, :, :]

        # 
        smoothed_stack = gaussian_filter(temp, sigma)

        #
        temp = temp.mean(0)

        # Plot the slice
        fig, ax = plt.subplots()
        ax.imshow(temp, cmap='gray')

        plt.text(x=10, y=10, 
                 s=str(i/30)+ " sec", color='white', fontsize=20, 
                 ha='left', va='top', alpha=0.7)

        # Save the current slice to a temporary .png file
        plt.savefig('temp.png')

        # Read the temporary .png file and add it to the movie
        writer.append_data(imageio.imread('temp.png'))

        # Close the figure to free up memory
        plt.close(fig)

        #
        #if i>500:
        #    break

    # Close the writer
    writer.close()

  0%|          | 0/334 [00:00<?, ?it/s]<ipython-input-3-a47507d91e8f>:72: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread('temp.png'))
100%|██████████| 334/334 [01:22<00:00,  4.03it/s]


In [9]:
#############################################
######### LOAD SUITE2P BINARIZED DATA #######
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210214/suite2p/plane0/'
# data_dir = '/media/cat/4TB/donato/nathalie/plane0'
# data_dir = '/media/cat/4TB/donato/renan/renan'
#data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
data_dir = '/media/cat/4TB/donato/DON-006084/20210520/'

data_dir = '/media/cat/4TB/donato/andres/DON011733/20230118/calibration/'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True                          # outputs additional information during processing
c.recompute_binarization = False           # recomputes binarization and other processing steps; False: loads from previous saved locations if avialable
c.data_dir = data_dir
c.load_suite2p()                          # this function assumes output dirs is either in data_dir OR data_dir/suite2p/plane0/
c.save_matlab = False
#
c.detrend_model_order = 1

#
c.save_python = True

#
c.load_binarization()
traces = c.F_upphase_bin    # c.F_upphase_bin

print ("binarized data (# cells, # time points): ", traces.shape)


  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (93, 20000)
         self.Fneu (neuropile):  (93, 20000)
         self.iscell (Suite2p cell classifier output):  (306, 2)
              of which number of good cells:  (93,)
         self.spks (deconnvoved spikes):  (93, 20000)
         self.stat (footprints structure):  (93,)
         mean std over all cells :  57.0
   todo: print binarization defaults...
binarized data (# cells, # time points):  (93, 20000)


In [10]:
#################################################
############# LOAD CONTOURS ###################
#################################################

# this loads the suite2p footprints
c.load_footprints()
print ("# of footprints; ", len(c.footprints))

# these should be the contours that we can plot
print ("# of contours: ", len(c.contours))

plt.figure()
for k in range(len(c.footprints)):
    plt.plot(c.contours[k][:,0], c.contours[k][:,1])
plt.xlim(0,512)
plt.ylim(0,512)
plt.show()

# of footprints;  93
# of contours:  93


In [4]:
#################################################
############# LOAD TRACES FOR ROIS ##############
#################################################

print ("# of traces: ", len(c.F_filtered))

plt.figure()
plt.plot(c.F_filtered[0])
plt.show()

# of traces:  155
